# EDA y Feature Engineering para predecir **SalePrice** (Ames Housing)
_Generado automáticamente el 2025-09-03 20:00:49_


Este notebook realiza:
1) Carga de datos y verificación básica  
2) Análisis exploratorio (distribuciones, valores faltantes, correlaciones)  
3) Ingeniería de características basada en sentido común del dominio  
4) Preparación de **pipelines** con `ColumnTransformer`  
5) Entrenamiento y evaluación de un modelo de **Regresión Lineal** (y variantes Ridge/Lasso opcionales)

**Archivos esperados** (en el mismo entorno):
- `/mnt/data/train.csv` — dataset de entrenamiento
- `/mnt/data/data_description.txt` — diccionario de variables (opcional para consulta)

> Nota: evita fijar estilos o colores en los gráficos para mantener consistencia con las directrices de visualización.


## 1) Setup y carga de datos

In [4]:
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

DATA_PATH = "data/train.csv"
DESC_PATH = "data/data_description.txt"


df = pd.read_csv(DATA_PATH)

print(df.shape)
df.head()


(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 2) Inspección rápida

In [5]:

display(df.info())
display(df.describe(include='all').T.head(30))

# Valores faltantes
na = df.isna().sum().sort_values(ascending=False)
na = na[na>0]
na.to_frame("missing_count").head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Id,1460.0,NaN,NaN,NaN,730.5,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,NaN,NaN,NaN,56.89726,42.300571,20.0,20.0,50.0,70.0,190.0
MSZoning,1460,5,RL,1151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LotFrontage,1201.0,NaN,NaN,NaN,70.049958,24.284752,21.0,59.0,69.0,80.0,313.0
LotArea,1460.0,NaN,NaN,NaN,10516.828082,9981.264932,1300.0,7553.5,9478.5,11601.5,215245.0
Street,1460,2,Pave,1454,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alley,91,2,Grvl,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LotShape,1460,4,Reg,925,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LandContour,1460,4,Lvl,1311,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Utilities,1460,2,AllPub,1459,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,missing_count
PoolQC,1453
MiscFeature,1406
Alley,1369
Fence,1179
MasVnrType,872
FireplaceQu,690
LotFrontage,259
GarageQual,81
GarageFinish,81
GarageType,81


## 3) Target: distribución de `SalePrice` y log-transform

In [ ]:

fig = plt.figure()
df['SalePrice'].hist(bins=40)
plt.title("Distribución de SalePrice")
plt.xlabel("SalePrice")
plt.ylabel("Frecuencia")
plt.show()

df['LogSalePrice'] = np.log1p(df['SalePrice'])

fig = plt.figure()
df['LogSalePrice'].hist(bins=40)
plt.title("Distribución de Log(SalePrice+1)")
plt.xlabel("LogSalePrice")
plt.ylabel("Frecuencia")
plt.show()

df[['SalePrice','LogSalePrice']].describe().T


## 4) Correlaciones numéricas con `SalePrice`

In [ ]:

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr = df[num_cols].corr(numeric_only=True)['SalePrice'].sort_values(ascending=False)
corr.head(20)


## 5) Limpieza básica y mapeos de categorías ordinales

In [ ]:

df_clean = df.copy()

# Variables con 'NA' que significa 'sin' (no es missing real) — establecer a 'None' antes de imputación
none_cats = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
             'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
for c in none_cats:
    if c in df_clean.columns:
        df_clean[c] = df_clean[c].fillna('None')

# Mapas ordinales de calidad (Ex > Gd > TA > Fa > Po)
qual_map = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'None':0}
for c in ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual','FireplaceQu','GarageQual','GarageCond','PoolQC']:
    if c in df_clean.columns:
        df_clean[c] = df_clean[c].map(qual_map).fillna(0).astype(int)

# 'BsmtExposure' tiene categorías cualitativas: Gd > Av > Mn > No > None
exp_map = {'Gd':3,'Av':2,'Mn':1,'No':0,'None':0}
if 'BsmtExposure' in df_clean.columns:
    df_clean['BsmtExposure'] = df_clean['BsmtExposure'].map(exp_map).fillna(0).astype(int)

# 'Functional' (mejor = Typ)
func_map = {'Typ':6,'Min1':5,'Min2':4,'Mod':3,'Maj1':2,'Maj2':1,'Sev':0,'Sal':0}
if 'Functional' in df_clean.columns:
    df_clean['Functional'] = df_clean['Functional'].map(func_map).fillna(func_map['Typ']).astype(int)

df_clean.head()


## 6) Feature engineering

In [ ]:

df_fe = df_clean.copy()

# Superficies
for col in ['TotalBsmtSF','1stFlrSF','2ndFlrSF']:
    if col not in df_fe.columns:
        raise KeyError(f"Falta la columna {col} para TotalSF")
df_fe['TotalSF'] = df_fe['TotalBsmtSF'] + df_fe['1stFlrSF'] + df_fe['2ndFlrSF']

# Baños totales (ponderación estándar: 0.5 para medios baños)
df_fe['TotalBath'] = (
    df_fe.get('FullBath',0) + 0.5*df_fe.get('HalfBath',0) +
    df_fe.get('BsmtFullBath',0) + 0.5*df_fe.get('BsmtHalfBath',0)
)

# Edades relativas al año de venta
if {'YrSold','YearBuilt'}.issubset(df_fe.columns):
    df_fe['HouseAge'] = df_fe['YrSold'] - df_fe['YearBuilt']
if {'YrSold','YearRemodAdd'}.issubset(df_fe.columns):
    df_fe['SinceRemodel'] = df_fe['YrSold'] - df_fe['YearRemodAdd']
if {'YrSold','GarageYrBlt'}.issubset(df_fe.columns):
    df_fe['GarageAge'] = np.where(df_fe['GarageYrBlt'].isna(), np.nan, df_fe['YrSold'] - df_fe['GarageYrBlt'])

# Indicadores binarios de presencia
df_fe['HasPool'] = (df_fe.get('PoolArea',0) > 0).astype(int)
df_fe['HasGarage'] = (~df_fe.get('GarageType',"None").isin(['None', np.nan])).astype(int) if 'GarageType' in df_fe.columns else 0
df_fe['HasFireplace'] = (df_fe.get('Fireplaces',0) > 0).astype(int)
df_fe['HasPorch'] = ((df_fe.get('OpenPorchSF',0)+df_fe.get('EnclosedPorch',0)+df_fe.get('3SsnPorch',0)+df_fe.get('ScreenPorch',0)) > 0).astype(int)

# Target log
df_fe['LogSalePrice'] = np.log1p(df_fe['SalePrice'])

df_fe[['SalePrice','LogSalePrice','TotalSF','TotalBath','HouseAge','SinceRemodel','GarageAge','HasPool','HasGarage','HasFireplace','HasPorch']].head()


## 7) Separación de variables y columnas por tipo

In [ ]:

target = 'SalePrice'  # también se usará LogSalePrice para el modelo
y = df_fe['LogSalePrice']  # mejor comportamiento lineal

# Selección de características: usa todo excepto identificadores/target
drop_cols = ['Id','SalePrice','LogSalePrice']
X = df_fe.drop(columns=[c for c in drop_cols if c in df_fe.columns])

num_features = X.select_dtypes(include=[np.number]).columns.tolist()
cat_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

len(num_features), len(cat_features), num_features[:5], cat_features[:5]


## 8) Pipeline de preprocesamiento y modelos

In [ ]:

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=True, with_std=True))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features),
    ]
)

models = {
    "Linear": LinearRegression(),
    "Ridge(alpha=10)": Ridge(alpha=10.0, random_state=42),
    "Lasso(alpha=0.001)": Lasso(alpha=0.001, random_state=42, max_iter=10000),
}

results = {}
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    # scoring: RMSE sobre SalePrice logarítmico
    neg_rmse = cross_val_score(pipe, X, y, scoring="neg_root_mean_squared_error", cv=kf)
    results[name] = {
        "RMSE_log_mean": -neg_rmse.mean(),
        "RMSE_log_std":  neg_rmse.std()
    }

pd.DataFrame(results).T.sort_values("RMSE_log_mean")


## 9) Entrenamiento final y coeficientes del modelo lineal

In [ ]:

pipe_lin = Pipeline(steps=[("preprocess", preprocess), ("model", LinearRegression())])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

pipe_lin.fit(X_train, y_train)
preds = pipe_lin.predict(X_valid)

rmse = mean_squared_error(y_valid, preds, squared=False)
r2 = r2_score(y_valid, preds)

print(f"Valid RMSE (log): {rmse:.4f}")
print(f"Valid R^2     : {r2:.4f}")

# Extraer nombres de características después del one-hot
feature_names_num = num_features
feature_names_cat = list(pipe_lin.named_steps['preprocess']\
                         .named_transformers_['cat']\
                         .named_steps['onehot']\
                         .get_feature_names_out(cat_features))
feature_names = feature_names_num + feature_names_cat

coefs = pipe_lin.named_steps['model'].coef_
coef_df = pd.DataFrame({"feature": feature_names, "coef": coefs}).sort_values("coef", ascending=False)
coef_df.head(20)


## 10) (Opcional) Guardar artefactos

In [ ]:

# Guardar dataset enriquecido y coeficientes, si se desea
df_fe.to_csv("/mnt/data/train_fe.csv", index=False)
coef_df.to_csv("/mnt/data/linear_model_coefs.csv", index=False)
print("Guardados: /mnt/data/train_fe.csv y /mnt/data/linear_model_coefs.csv")


## Apéndice: Diccionario de variables (opcional)

In [ ]:

if os.path.exists(DESC_PATH):
    with open(DESC_PATH, 'r', encoding='utf-8', errors='ignore') as f:
        text = f.read()
    print(text[:2000] + "\n...\n[Truncado]")
else:
    print("No se encontró data_description.txt")
